short summary:
----
This is a notebook that's for performance testing. On Windows through WSL, it turns out to be at minimum taking two times the time to run as compared to a Linux kernel. This is just on the CPU side.

In [1]:
!whoami

llinfeng


In [2]:
# System Hardware Information: CPU and GPU
import platform
import subprocess
import sys

print("=" * 80)
print("CPU INFORMATION")
print("=" * 80)
print(f"Processor: {platform.processor()}")
print(f"Machine: {platform.machine()}")
print(f"Platform: {platform.platform()}")

# Get detailed CPU info using /proc/cpuinfo
try:
    with open('/proc/cpuinfo', 'r') as f:
        cpuinfo = f.read()
        # Extract model name
        for line in cpuinfo.split('\n'):
            if 'model name' in line.lower():
                print(f"CPU Model: {line.split(':')[1].strip()}")
                break
        # Count physical cores
        physical_cores = len(set([line for line in cpuinfo.split('\n') if 'physical id' in line.lower()]))
        if physical_cores == 0:
            physical_cores = 1
        # Count cores per CPU
        cores_per_cpu = len(set([line for line in cpuinfo.split('\n') if 'core id' in line.lower()]))
        if cores_per_cpu == 0:
            cores_per_cpu = len([line for line in cpuinfo.split('\n') if 'processor' in line.lower()])
        total_cores = physical_cores * cores_per_cpu
        # Count threads
        threads = len([line for line in cpuinfo.split('\n') if line.startswith('processor')])
        print(f"Physical CPUs: {physical_cores}")
        print(f"Cores per CPU: {cores_per_cpu}")
        print(f"Total Physical Cores: {total_cores}")
        print(f"Total Threads/Logical CPUs: {threads}")
except Exception as e:
    print(f"Could not read CPU info: {e}")

# Try using psutil if available
try:
    import psutil
    print(f"\nCPU Frequency: {psutil.cpu_freq().current:.2f} MHz (max: {psutil.cpu_freq().max:.2f} MHz)")
    print(f"CPU Usage: {psutil.cpu_percent(interval=1)}%")
    print(f"CPU Count (logical): {psutil.cpu_count(logical=True)}")
    print(f"CPU Count (physical): {psutil.cpu_count(logical=False)}")
except ImportError:
    print("\n(Install 'psutil' for more detailed CPU information: pip install psutil)")
except Exception as e:
    print(f"\nError getting CPU info with psutil: {e}")

print("\n" + "=" * 80)
print("GPU INFORMATION")
print("=" * 80)

# Check for NVIDIA GPUs using nvidia-smi
try:
    result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free,driver_version,compute_cap', 
                            '--format=csv,noheader,nounits'], 
                           capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        gpu_lines = result.strip().split('\n')
        for i, line in enumerate(gpu_lines):
            if line.strip():
                parts = [p.strip() for p in line.split(',')]
                if len(parts) >= 4:
                    print(f"\nGPU {i}:")
                    print(f"  Name: {parts[0]}")
                    print(f"  Total Memory: {parts[1]} MB")
                    print(f"  Free Memory: {parts[2]} MB")
                    print(f"  Driver Version: {parts[3]}")
                    if len(parts) >= 5:
                        print(f"  Compute Capability: {parts[4]}")
    else:
        print("nvidia-smi command failed or no NVIDIA GPU detected")
except FileNotFoundError:
    print("nvidia-smi not found. NVIDIA drivers may not be installed.")
except subprocess.TimeoutExpired:
    print("nvidia-smi command timed out")
except Exception as e:
    print(f"Error running nvidia-smi: {e}")

# Check PyTorch GPU
try:
    import torch
    print(f"\nPyTorch Version: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"CUDA Available: Yes")
        print(f"CUDA Version: {torch.version.cuda}")
        print(f"cuDNN Version: {torch.backends.cudnn.version()}")
        print(f"Number of GPUs: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"\nGPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"  Memory Allocated: {torch.cuda.memory_allocated(i) / 1024**3:.2f} GB")
            print(f"  Memory Reserved: {torch.cuda.memory_reserved(i) / 1024**3:.2f} GB")
            print(f"  Total Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")
    else:
        print("CUDA Available: No")
except ImportError:
    print("\n(PyTorch not installed - install with: pip install torch)")
except Exception as e:
    print(f"\nError checking PyTorch GPU: {e}")

# Check TensorFlow GPU
try:
    import tensorflow as tf
    print(f"\nTensorFlow Version: {tf.__version__}")
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print(f"TensorFlow GPUs Detected: {len(gpus)}")
        for i, gpu in enumerate(gpus):
            print(f"\nGPU {i}: {gpu.name}")
            try:
                details = tf.config.experimental.get_device_details(gpu)
                if details:
                    print(f"  Details: {details}")
            except:
                pass
    else:
        print("TensorFlow GPUs Detected: 0")
except ImportError:
    print("\n(TensorFlow not installed - install with: pip install tensorflow)")
except Exception as e:
    print(f"\nError checking TensorFlow GPU: {e}")

print("\n" + "=" * 80)

CPU INFORMATION
Processor: x86_64
Machine: x86_64
Platform: Linux-6.14.0-37-generic-x86_64-with-glibc2.39
CPU Model: 11th Gen Intel(R) Core(TM) i9-11900KF @ 3.50GHz
Physical CPUs: 1
Cores per CPU: 8
Total Physical Cores: 8
Total Threads/Logical CPUs: 16

CPU Frequency: 1316.46 MHz (max: 5150.00 MHz)
CPU Usage: 1.6%
CPU Count (logical): 16
CPU Count (physical): 8

GPU INFORMATION
Error running nvidia-smi: 'CompletedProcess' object has no attribute 'strip'

(PyTorch not installed - install with: pip install torch)


2025-12-15 09:38:34.208576: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-15 09:38:34.231269: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-15 09:38:34.231292: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-15 09:38:34.231883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-15 09:38:34.235792: I tensorflow/core/platform/cpu_feature_guar


TensorFlow Version: 2.15.1
TensorFlow GPUs Detected: 1

GPU 0: /physical_device:GPU:0
  Details: {'compute_capability': (8, 6), 'device_name': 'NVIDIA GeForce RTX 3090'}



2025-12-15 09:38:35.236673: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-15 09:38:35.259272: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-15 09:38:35.261928: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# EPF Toolbox: forecasting models demo

Second notebook to generate day-ahead forecasts with `epftoolbox.models`.
- Mirrors the examples in `examples/` and the [models documentation](https://epftoolbox.readthedocs.io/en/latest/modules/models.html).
- Reuses data cached in `datasets/` from `01_load_data.ipynb`.
- Saves forecasts to `forecasts_local/` (created below).


In [3]:
from pathlib import Path

repo_root = Path.cwd().resolve()
if repo_root.name == 'Notebooks':
    repo_root = repo_root.parent

data_path = repo_root / 'datasets'
work_path = repo_root / 'forecasts_local'
work_path.mkdir(exist_ok=True)

print(f'Repo root: {repo_root}')
print(f'Data path: {data_path}')
print(f'Experiments path: {work_path}')


Repo root: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox
Data path: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/datasets
Experiments path: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/forecasts_local


# Verify GPU compute
In Temrinal, run: 
```bash
watch -n0.2 nvidia-smi  
```

In [4]:
import sys, os

print("sys.executable:", sys.executable)
print("sys.prefix:", sys.prefix)
print("VIRTUAL_ENV:", os.environ.get("VIRTUAL_ENV"))

sys.executable: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/.venv/bin/python3
sys.prefix: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/.venv
VIRTUAL_ENV: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/.venv


In [5]:
import tensorflow as tf
print(tf.config.list_physical_devices("GPU"))
with tf.device("/GPU:0"):
     a = tf.random.normal([2000, 2000])
     b = tf.random.normal([2000, 2000])
     _ = tf.matmul(a, b)  # warms up JIT

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-12-15 09:38:35.382338: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-15 09:38:35.385834: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-15 09:38:35.389688: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
import tensorflow as tf, time

tf.debugging.set_log_device_placement(True)  # optional, see placements

with tf.device("/GPU:0"):
    a = tf.random.normal([8192, 8192])
    b = tf.random.normal([8192, 8192])
    for i in range(3):
        t0 = time.time()
        c = tf.matmul(a, b)
        _ = c.numpy()  # force materialization
        print(f"iter {i} took {time.time() - t0:.3f}s")

iter 0 took 0.155s
iter 1 took 0.115s
iter 2 took 0.114s


## LEAR daily recalibration (CPU extensive)

Quick run of the LEAR example (`epftoolbox.models.evaluate_lear_in_test_dataset`) to produce a forecast CSV in `forecasts_local/`.


In [ ]:
%%time
# 6 hours of run time completed on T14 | 546 minutes didn't finish at all on WSL (P14s)
# 109m on AW machine (i9-11900KF)
from epftoolbox.models import evaluate_lear_in_test_dataset

dataset = 'PJM'
years_test = 2
calibration_window = 364 * 3

forecast_lear = evaluate_lear_in_test_dataset(
    path_datasets_folder=str(data_path),
    path_recalibration_folder=str(work_path),
    dataset=dataset,
    years_test=years_test,
    calibration_window=calibration_window,
)

forecast_lear.head()


/home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/.venv/lib/python3.11/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


2016-12-27 - sMAPE: 5.99%  |  MAE: 1.604
2016-12-28 - sMAPE: 4.67%  |  MAE: 1.298
2016-12-29 - sMAPE: 4.51%  |  MAE: 1.274
2016-12-30 - sMAPE: 5.07%  |  MAE: 1.451
2016-12-31 - sMAPE: 6.30%  |  MAE: 1.682
2017-01-01 - sMAPE: 8.03%  |  MAE: 2.036
2017-01-02 - sMAPE: 7.94%  |  MAE: 2.016
2017-01-03 - sMAPE: 7.90%  |  MAE: 2.009
2017-01-04 - sMAPE: 7.64%  |  MAE: 1.971
2017-01-05 - sMAPE: 7.65%  |  MAE: 2.063
2017-01-06 - sMAPE: 7.37%  |  MAE: 2.030
2017-01-07 - sMAPE: 7.33%  |  MAE: 2.063
2017-01-08 - sMAPE: 8.02%  |  MAE: 2.352
2017-01-09 - sMAPE: 8.89%  |  MAE: 2.639
2017-01-10 - sMAPE: 8.96%  |  MAE: 2.629
2017-01-11 - sMAPE: 9.45%  |  MAE: 2.693
2017-01-12 - sMAPE: 9.29%  |  MAE: 2.630
2017-01-13 - sMAPE: 9.07%  |  MAE: 2.576
2017-01-14 - sMAPE: 8.96%  |  MAE: 2.549
2017-01-15 - sMAPE: 8.95%  |  MAE: 2.537
2017-01-16 - sMAPE: 8.73%  |  MAE: 2.482
2017-01-17 - sMAPE: 8.82%  |  MAE: 2.504
2017-01-18 - sMAPE: 8.72%  |  MAE: 2.475
2017-01-19 - sMAPE: 8.70%  |  MAE: 2.479
2017-01-20 - sMA

,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2016-12-27,19.100844,18.264595,17.53075,17.434241,18.22411,19.853216,23.788848,25.228666,26.251533,27.419878,...,26.20889,26.338958,28.022928,35.886231,32.565618,31.739757,31.986766,28.902147,24.976016,22.516413
2016-12-28,21.947657,21.407247,20.915056,20.843612,21.378066,23.707062,30.277734,33.646398,31.340376,30.944352,...,25.61739,26.071015,27.769258,35.816337,36.297189,34.476443,32.25681,29.718951,25.991414,24.034067
2016-12-29,21.963601,20.970229,20.101968,20.117763,20.486912,22.704496,28.13348,32.937301,30.116596,31.026365,...,28.609186,28.888347,30.637242,38.876011,36.60512,35.55204,35.255296,30.515633,26.571027,24.230899
2016-12-30,22.895867,21.905473,21.212615,21.177958,21.596962,23.675649,28.202682,32.008085,31.970438,33.674966,...,29.523996,29.002559,31.157213,39.438211,36.56299,35.710511,36.192968,32.226899,28.493623,25.891891
2016-12-31,23.020518,21.974983,21.226,20.678976,20.496707,21.093947,22.753666,24.696514,26.168603,26.245395,...,17.416643,17.177852,18.647874,28.98324,24.530161,20.954127,22.603399,23.652396,22.838756,21.804186


## DNN daily recalibration (requires hyperparameters)

The DNN helpers mirror `examples/recalibrating_dnn_simplified.py`, but they need a trials file from `examples/optimizing_hyperparameters_dnn.py`.
The cell below checks for the expected file in `forecasts_local/` and only runs when it exists.


### Optional: create DNN hyperparameter trials

The DNN helper needs a trials file (hyperparameter search). If it is missing, run a small hyperopt sweep here to generate it. Increase `max_evals` for a proper search; the default keeps this demo quick.


In [8]:
# 8 min run time
from epftoolbox.models import hyperparameter_optimizer

nlayers = 2
shuffle_train = 1
data_augmentation = 0
calibration_window = 4
years_test = 2
dataset = 'PJM'
experiment_id = 1
begin_test_date = None
end_test_date = None
max_evals = 10  # increase for a full search; small for a quick demo

trials_file_name = (
    f'DNN_hyperparameters_nl{nlayers}_dat{dataset}_YT{years_test}'
    f"{'_SF' if shuffle_train else ''}"
    f"{'_DA' if data_augmentation else ''}"
    f'_CW{calibration_window}_{experiment_id}'
)
trials_path = work_path / trials_file_name

if trials_path.exists():
    print('Trials file already exists, skipping hyperparameter search:')
    print(trials_path)
else:
    print('Running a small hyperparameter search to create trials file...')
    hyperparameter_optimizer(
        path_datasets_folder=str(data_path),
        path_hyperparameters_folder=str(work_path),
        new_hyperopt=True,
        max_evals=max_evals,
        nlayers=nlayers,
        dataset=dataset,
        years_test=years_test,
        calibration_window=calibration_window,
        shuffle_train=bool(shuffle_train),
        data_augmentation=bool(data_augmentation),
        experiment_id=experiment_id,
        begin_test_date=begin_test_date,
        end_test_date=end_test_date,
    )
    print('Saved trials to:', trials_path)


Trials file already exists, skipping hyperparameter search:
/home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/forecasts_local/DNN_hyperparameters_nl2_datPJM_YT2_SF_CW4_1


In [ ]:
%%time
# 6 hours of run time on i7-1260P + 3080 (10GB, eGPU)
# 276m on AW machine (i9-11900KF, 3090, 24GB)
from epftoolbox.models import evaluate_dnn_in_test_dataset

nlayers = 2
shuffle_train = 1
data_augmentation = 0
calibration_window = 4
years_test = 2
dataset = 'PJM'
experiment_id = 1
begin_test_date = None
end_test_date = None

trials_file_name = (
    f'DNN_hyperparameters_nl{nlayers}_dat{dataset}_YT{years_test}'
    f"{'_SF' if shuffle_train else ''}"
    f"{'_DA' if data_augmentation else ''}"
    f'_CW{calibration_window}_{experiment_id}'
)
trials_path = work_path / trials_file_name

if trials_path.exists():
    forecast_dnn = evaluate_dnn_in_test_dataset(
        experiment_id=experiment_id,
        path_hyperparameter_folder=str(work_path),
        path_datasets_folder=str(data_path),
        path_recalibration_folder=str(work_path),
        nlayers=nlayers,
        dataset=dataset,
        years_test=years_test,
        shuffle_train=bool(shuffle_train),
        data_augmentation=bool(data_augmentation),
        calibration_window=calibration_window,
        new_recalibration=True,
        begin_test_date=begin_test_date,
        end_test_date=end_test_date,
    )
    display(forecast_dnn.head())
else:
    print('Skipping DNN run – hyperparameter trials file not found:')
    print(trials_path)
    print('Run examples/optimizing_hyperparameters_dnn.py to generate it.')

2025-12-15 11:28:33.827062: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-12-15 11:28:34.668368: I external/local_xla/xla/service/service.cc:168] XLA service 0x74ca54f7d550 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-15 11:28:34.668384: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-12-15 11:28:34.671634: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-15 11:28:34.694504: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1765769314.745805  195903 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2016-12-27 - sMAPE: 2.72%  |  MAE: 0.662
2016-12-28 - sMAPE: 2.81%  |  MAE: 0.756


2016-12-29 - sMAPE: 4.07%  |  MAE: 1.130


2016-12-30 - sMAPE: 3.66%  |  MAE: 1.030


2016-12-31 - sMAPE: 4.45%  |  MAE: 1.241
2017-01-01 - sMAPE: 4.36%  |  MAE: 1.208


...

2018-12-21 - sMAPE: 13.68%  |  MAE: 3.622


2018-12-22 - sMAPE: 13.67%  |  MAE: 3.619


2018-12-23 - sMAPE: 13.65%  |  MAE: 3.615
2018-12-24 - sMAPE: 13.65%  |  MAE: 3.613


,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,...,h14,h15,h16,h17,h18,h19,h20,h21,h22,h23
Date,,,,,,,,,,,,,,,,,,,,,
2016-12-27,19.200207,18.098473,17.03019,16.508541,17.20569,19.139465,22.312382,24.231153,24.674744,25.279732,...,23.878384,24.048077,25.643993,31.250399,29.95454,28.132656,27.241764,25.427444,22.992561,20.914829
2016-12-28,21.709909,21.104456,20.37225,19.906668,20.531923,22.197878,27.611332,30.827711,30.454529,30.690544,...,26.006838,25.606283,27.296911,34.285515,35.019772,33.122585,31.276649,28.377609,25.472246,23.579344
2016-12-29,21.995098,21.148149,20.378672,20.117315,20.532421,22.48793,28.599201,31.856134,30.331537,30.027437,...,24.395754,23.971613,25.384821,32.715721,32.669174,30.694637,28.998264,26.77943,24.180956,22.472441
2016-12-30,22.453262,21.371069,20.392267,20.035448,20.50878,22.297714,27.360678,30.532217,30.16576,30.747328,...,27.100452,26.993523,28.668613,35.800022,35.224594,33.065804,32.124306,29.503237,26.161888,23.856985
2016-12-31,22.848526,21.752214,21.101294,20.580898,20.801155,21.881824,25.413719,28.148069,28.941587,29.648863,...,26.318247,26.281195,27.96726,34.990414,33.848492,32.030037,31.34783,28.612736,25.830736,23.711338


CPU times: user 4h 34min 12s, sys: 36min 18s, total: 5h 10min 30s
Wall time: 4h 36min 2s
